In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo

from sklearn.preprocessing import MinMaxScaler


In [11]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Function to center, scale, and return a series
def minmax_scale_group(group):
    median = group.median()
    centered_values = group - median
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(centered_values.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    medians[group.name] = median  # Store the median for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_minmax_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    median = medians[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values + median
    return pd.Series(original_values, index=group.index, name=group_name)

# Custom metric function
def multinacional_metric(y_true_scaled, y_pred_scaled):
    # Inverse transform and decenter the predicted values
    y_pred_original = y_pred_scaled.groupby(y_pred_scaled.index).apply(inverse_minmax_scale_group, group_keys=False)
    y_true_original = y_true_scaled.groupby(y_true_scaled.index).apply(inverse_minmax_scale_group, group_keys=False)
    # Calculate the metric using true values (already in the original scale)
    metric = abs(sum(y_true_original - y_pred_original)) / sum(y_true_original)
    
    return metric




In [12]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final = pd.read_parquet(DATOS_DIR+'/FE_02_dataset.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [13]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-11-01']


#Filtro de no compradores
#Step 1: Ensure the index is a datetime type
df_final.index = df_final.index.to_timestamp()
# Step 2: Determine the last date and calculate the date 3 months prior
ls_date  = df_final.index.max()
three_months_prior = ls_date - pd.DateOffset(months=3)

# Step 3: Filter the dataframe to include onl+y rows within the last 3 months
last_3_months_df = df_final[df_final.index >= three_months_prior]

# Step 4: Identify the unique client_id that have purchased within this period
active_clients = last_3_months_df['customer_id'].unique()

# Step 5: Filter the original dataframe to include only these client_id
df_final = df_final[df_final['customer_id'].isin(active_clients)]

df_final.index = pd.PeriodIndex(df_final.index, freq='M')


#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]

In [14]:
df_final.index.max()
#df_final['tn_2'].fillna(0, inplace=True)

Period('2019-11', 'M')

In [15]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    ### Agrupar y escalar
    scalers = {}
    medians = {}
    df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
    df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse')
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    last_data_points = df_producto[df_producto.index == df_producto.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month(model, last_data_points)
    preds = predictions.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20524
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[35]	validation's rmse: 0.022655


C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	validation's rmse: 0.0320158
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[32]	validation's rmse: 0.0528865
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[45]	validation's rmse: 0.0116402
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[48]	validation's rmse: 0.00620295
Overall rmse metric:  3.84766007646618e-05
             tn
20524  5.497266
Fitting and predicting for product_id: 20311
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[31]	validation's rmse: 0.0427303


C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[41]	validation's rmse: 0.0329177
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[41]	validation's rmse: 0.0366298
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	validation's rmse: 0.0170406
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[30]	validation's rmse: 0.0117244
Overall rmse metric:  0.0001374616905689874
              tn
20311  20.092488
Fitting and predicting for product_id: 20654
Training until validation scores don't improve for 10 rounds


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[69]	validation's rmse: 0.0096162
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	validation's rmse: 0.0031649
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[161]	validation's rmse: 0.142048
Overall rmse metric:  1.0016568191998897e-05
             tn
20995  3.228891
Fitting and predicting for product_id: 20210
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 0.0460277
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[25]	validation's rmse: 0.172024


C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	validation's rmse: 0.123652
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	validation's rmse: 0.0584761
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[11]	validation's rmse: 0.0327989
Overall rmse metric:  0.001075765486984057
               tn
20210  133.319501
Fitting and predicting for product_id: 20127
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	validation's rmse: 0.123953
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	validation's rmse: 0.0542946
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[33]	validation's rmse: 0.0387227


C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn'] = df_producto.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
C:\Users\Admin\AppData\Local\Temp\ipykernel_3024\3720373635.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_producto['tn_2'] = df_producto.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[38]	validation's rmse: 0.165441
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[27]	validation's rmse: 0.0276448
Overall rmse metric:  0.0007642372406176097
              tn
20127  449.88825


In [16]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0014-prediccion-rmse_scaled_CORRECT-product_id-No_Buyers.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)
print("Error prediccion 12-19: " , 
      abs(sum(df_true.groupby('product_id')['tn'].sum() - predictions_all['tn']))/sum(df_true.groupby('product_id')['tn'].sum()))

Overall custom metric:  0.0007642372406176097
Error prediccion 12-19:  0.24135355899309321
